# Explore here

In [2]:
import requests
import zipfile
import io

url = "https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip"
response = requests.get(url)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("dogs-vs-cats")

------------------------------------------------------

In [4]:
import os
import shutil

images_folder = "dogs-vs-cats/dogs-vs-cats/train"
source_folder = "data/processed"
cats_folder = "data/processed/cats"
dogs_folder = "data/processed/dogs"

os.makedirs(source_folder, exist_ok=True)
os.makedirs(cats_folder, exist_ok=True)
os.makedirs(dogs_folder, exist_ok=True)

cat_count = 0
dog_count = 0
max_images = 100

for filename in os.listdir(images_folder):
    file_path = os.path.join(images_folder, filename)
    if os.path.isfile(file_path):
        nombre = filename.lower()
        if nombre.startswith("cat") and cat_count < max_images:
            shutil.copy(file_path, os.path.join(cats_folder, filename))
            cat_count += 1
        elif nombre.startswith("dog") and dog_count < max_images:
            shutil.copy(file_path, os.path.join(dogs_folder, filename))
            dog_count += 1

        if cat_count >= max_images and dog_count >= max_images:
            break

In [5]:
from keras.preprocessing import image
import numpy as np
import os

def load_and_preprocess_images(data_dir, target_size=(224, 224)):
    images = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                img_path = os.path.join(label_dir, filename)
                try:
                    img = image.load_img(img_path, target_size=target_size)
                    img_array = image.img_to_array(img)
                    img_array /= 255.0
                    images.append(img_array)

                    if label == "cats":
                        labels.append(0)
                    elif label == "dogs":
                        labels.append(1)
                except Exception as e:
                    print(f"Error cargando la imagen {img_path}: {e}")

    return np.array(images), np.array(labels)


data_dir = "data/processed"
images, labels = load_and_preprocess_images(data_dir)

# Verifica cuántas se cargaron
print(f"Total images loaded: {images.shape}")
print(f"Total labels loaded: {labels.shape}")

Total images loaded: (200, 224, 224, 3)
Total labels loaded: (200,)


In [6]:
from sklearn.model_selection import train_test_split


if len(images) > 0:
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    print("Data split into training and testing sets.")
    print(f"Training data shape: {X_train.shape}")
    print(f"Testing data shape: {X_test.shape}")
    print(f"Training labels shape: {Y_train.shape}")
    print(f"Testing labels shape: {Y_test.shape}")
else:
    print("No images loaded to split.")

Data split into training and testing sets.
Training data shape: (160, 224, 224, 3)
Testing data shape: (40, 224, 224, 3)
Training labels shape: (160,)
Testing labels shape: (40,)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(images,labels, test_size=0.2, random_state=42)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)


print("Train image data generator created successfully.")

Train image data generator created successfully.


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow(X_test, Y_test, batch_size=32, shuffle=False)

print("Test image data generator created successfully.")

Test image data generator created successfully.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

model = Sequential()
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
# Note: The user's provided model snippet was cut off here.
# Assuming the model should end with Flatten and Dense layers for classification.
model.add(Flatten())
model.add(Dense(units=4096, activation="relu")) # Example Dense layer based on common VGG-like architectures
model.add(Dense(units=4096, activation="relu")) # Example Dense layer
model.add(Dense(units=1, activation="sigmoid")) # Output layer for binary classification

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   411,045,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 433,106,753 (1.61 GB)

 Trainable params: 433,106,753 (1.61 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model compiled successfully.")

Model compiled successfully.


In [11]:
# Train the model using the data generators
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,  # Number of batches per epoch
    epochs=2,  # You can adjust the number of epochs
    validation_data=test_generator,
    validation_steps=len(X_test) // 32 # Number of validation batches
)

print("Model training complete.")

Epoch 1/2


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 344s 67s/step - accuracy: 0.5540 - loss: 0.7857 - val_accuracy: 0.5625 - val_loss: 0.6860
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 385s 68s/step - accuracy: 0.4952 - loss: 0.6963 - val_accuracy: 0.5625 - val_loss: 0.6921
Model training complete.


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1
)

In [11]:
model.save('EPablos2013_deeplearning.keras')

In [1]:
from tensorflow.keras.models import load_model

model = load_model('EPablos2013_deeplearning.keras')

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 50 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Update this path to a valid image file in your environment
# Example: "data/processed/dogs/dog.1.jpg" or "data/processed/cats/cat.1.jpg"
source_image = "path/to/your/image.jpg" # Replace this with the actual path

# If the placeholder path is still there, try to find an example image
if source_image == "path/to/your/image.jpg":
    example_image_found = False
    for root, dirs, files in os.walk("data/processed"):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                source_image = os.path.join(root, file)
                example_image_found = True
                print(f"Using example image: {source_image}")
                break
        if example_image_found:
            break

    if not example_image_found:
        print("Could not find an example image in 'data/processed'. Please provide a valid path.")
        source_image = None # Set to None if no example image is found


if source_image:
    try:
        img = image.load_img(source_image, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Assuming 'model' is the loaded model from a previous cell
        if 'model' in locals():
            prediction = model.predict(img_array)

            if prediction[0][0] > 0.5:
                print("Dog")
            else:
                print("Cat")
        else:
            print("Model 'model' is not loaded. Please run the cell to load the model first.")

    except FileNotFoundError:
        print(f"Error: The file '{source_image}' was not found. Please provide a valid path to an image file.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("No source image path provided or found.")

Using example image: data/processed/cats/cat.10064.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
Dog
